<img src="squid2.jpg" width="200">
<h1><span style="Color: red">S</span>age <span style="Color: red">Qui</span>z <span style="Color: red">D</span>eveloper
</h1>

Welcome to Squid! An object-oriented approach to creating variants of quiz questions.

Squid was created by Florian Breuer (florianbreuer@gmail.com), inspired by code written by my colleague Björn Rüffer (https://bjoern.rueffer.info/).

This code can be found on Github: https://github.com/florianbreuer/Squid

First, we load the basic classes and tools:

In [ ]:
load("Squid-tools.sage")
import ipywidgets as widgets
from ipywidgets import Layout
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

Let's create a written-answer question. The question asks the student to invert a random 3x3 matrix. Each variant of this question will be an instance of a class we'll call <span style="font-family:Courier New; ">Invert3x3Matrix_written</span>, initialised with a 3x3 matrix A.

Most of the effort will go into typesetting the model solution, which is stored in 
<span style="font-family:Courier New; ">self.solution_text</span>.

After the class definition we include some code to display an example instance. 
<span style="font-family:Courier New; ">Q.test_solution_page()</span> writes the marking scheme to the file 
<span style="font-family:Courier New; ">test.tex</span>, which can then be compiled to check everything typesets nicely.

In [ ]:
# Linear Algebra: Invert 3x3 matrix, written-answer question

class Invert3x3Matrix_written(Question_Written):
    
    def __init__(self,A,variant_number=0):
        """A must be a 3x3 invertible matrix."""
        Question_Written.__init__(self)
        self.A = A
        if not A.is_invertible():
            raise ValueError('Matrix is not invertible')
        I = matrix(3,3,[1,0,0,0,1,0,0,0,1])  # identity matrix
        M = A.augment(I, subdivide=true)
        N = I.augment(A.inverse(), subdivide=true)
        
        self.question_text = "Compute the inverse of the following matrix, showing all your work:\n"+\
        "\\[ A = "+latex(A)+"\\]"
        
        self.solution_text = "We augment the matrix $A$ with the $3\\times 3$ identity matrix:\n"+\
        "\\[ [A|I] = "+latex(M)+"\\]\n"+\
        "Next we use row reduction to obtain the reduced row-echelon form of this augmented matrix:\n"+\
        "\\[ [A|I] \\sim "+latex(N)+"\\]\n"+\
        "Now we read off $A^{-1}$ as the right submatrix:\n"+\
        "\\[ A^{-1} = "+latex(A.inverse())+".\\]"
        
        # the table entries below are used to create a table of 
        # contents in the marking scheme.
        # In the case, the table contains only two columns, one for the 
        # matrix A, one for the inverse A^{-1}
        self.table_header = ['$A$', '$A^{-1}$']
        
        self.table_row = ['$'+latex(A)+'$', '$'+latex(A.inverse())+'$']
        
I = matrix(3,3,[1,0,0,0,1,0,0,0,1])

# We create a suitable matrix A by starting with the identity and performing some simple row operations:
A = scramble_full(I,c=1)
Q = Invert3x3Matrix_written(A)
display(Q)
Q.test_solution_page()



Once we're happy with the question, we create a list L of variants. We then write the questions to a file for uploading to Blackboard, and the marking scheme to .tex file. 

The marking scheme contains a table of all variants on the first page. If you find that this table is too long you may want to edit that by hand later.

In [ ]:
L=[]

for i in range(12):
    I = matrix(3,3,[1,0,0,0,1,0,0,0,1])
    A = scramble_full(I,c=1)
    display(A)
    L.append(Invert3x3Matrix_written(A))
    
SaveMarkingScheme(L, 'Invert3x3-MarkingScheme.tex')

SaveToBBfile(L, 'Invert3x3.txt')


Not using Blackboard? Just add some methods to 
<span style="font-family:Courier New; ">Question_Written</span> to output files for your favourite learning system, or to typeset paper exams for printing.

If you're not happy with the selection of variants (e.g. some variants may be too easy), then you can use the Widget-based variant selection tool below!

Here is another example (which makes use of the <span style="font-family:Courier New; ">linear_system</span> class):

In [ ]:
#LinearIndependent_Long
class LinearIndependent_Written(Question_Written):
    
    def __init__(self,A,variant_number=0):
        """A must be a 3x3 invertible matrix."""
        Question_Written.__init__(self)
        self.A = A
        if not A.is_invertible():
            raise ValueError('Matrix is not invertible')
        self.S = [matrix(v).transpose() for v in A.columns()]  # list of columns of A as 3x1 matrices
        self.variant_number = variant_number
        self.question_text = "Consider the collection of vectors\n"+\
        "\\[ S = \\left\\{"+", ".join([latex(v) for v in self.S])+"\\right\\}.\n\\]\n"+\
        "Is $S$ linearly independent? Carefully explain your answer.\n"
        C = var(['c1','c2','c3'])
        B = A.transpose()
        B = B.insert_row(3,[0,0,0])
        B = B.transpose()
        L = linear_system(B,['c1','c2','c3'])
        
        self.solution_text = "{\\bf Solution "+str(self.variant_number)+":} "+\
        "Suppose there are constants $c_1, c_2, c_3$ such that\n"+\
        "\\[\n"+" + ".join([latex(C[i])+latex(self.S[i]) for i in range(3)])+\
        r" = \left(\begin{array}{r}0\\0\\0\end{array}\right).\]"+"\n"+\
        "Then, equating both sides, we find that $c_1,c_2,c_3$ satisfy the linear equations\n"+L.latex_eqs()+"\n"+\
        "which has augmented matrix\n\\[\n"+L.latex_augmented_matrix()+\
        r"\sim\left[\begin{array}{ccc|c}1&0&0&0\\0&1&0&0\\0&0&1&0\end{array}\right]."+"\n\\]\n"+\
        "It follows that the only solution of the system is $c_1=c_2=c_3=0$. We conclude that $S$ is linearly independent."

        self.table_header = ['$S$']
        
        self.table_row = [r"$\left\{"+", ".join([latex(v) for v in self.S])+"\\right\\}$"]

# M = matrix.identity(3)
# A = scramble_full(M,2)      
# Q = LinearIndependent_Written(A)
# Q.test_solution_page()

L=[]

for i in range(20):
    M = matrix.identity(3)
    A = scramble_full(M,2)
    L.append(LinearIndependent_Written(A))
    
SaveMarkingScheme(L, 'LinearIndep-MarkingScheme.tex')

SaveToBBfile(L, 'LinearIndep.txt')

Let's make a multiple-choice question. This question asks for the directional derivative of the function $f$ in the direction $V$ at the point $P$. Each variant has a correct answer and 3 wrong answers. A "None of the above" option is added automatically.

We create a list of variants by iterating over lists of functions, vectors and points:

In [ ]:
x,y = var('x y')

class DirectionalDerivative(Question_MCQ):
    def __init__(self,f,V,P,variant_number=0):
        '''Compute the directional derivative of f(x,y) at the point P in the direction v.'''
        Question_Written.__init__(self)
        self.variant_number = variant_number
        
        fx = diff(f,x)
        fy = diff(f,y)
        
        u = (V[0]/sqrt(V[0]^2+V[1]^2),V[1]/sqrt(V[0]^2+V[1]^2))
        
        self.question_text = r"""Compute the directional derivative of \(f(x) = """\
        +latex(f)+\
        r"""\) at the point \(P = """\
        +latex(P)+\
        r"""\) in the direction \(\langle"""\
        +latex(V[0])+','+latex(V[1])+\
        r"""\rangle\)."""
        
        self.answer = r'\('+latex(u[0]*fx(x=P[0],y=P[1]) + u[1]*fy(x=P[0],y=P[1]))+r'\)'
        
        self.wrong_answers = [r'\('+latex(V[0]*fx(x=P[0],y=P[1]) + V[1]*fy(x=P[0],y=P[1]))+r'\)',
                              r'\('+latex(V[0]*fx(x=P[0],y=P[1]) - V[1]*fy(x=P[0],y=P[1]))+r'\)',
                              r'\('+latex(u[0]*fy(x=P[0],y=P[1]) + u[1]*fx(x=P[0],y=P[1]))+r'\)',
                             ]
        
# Q = DirectionalDerivative(x^2+y, (1,1), (-2,1))
# Q.has_distinct_answers()

flist = [x^2+y, x+y^2, x^3-y, y^3-x]
Plist = [(1,2),(1,-2),(-1,2),(2,1),(2,-1)]

L = [DirectionalDerivative(f,V,P) for f in flist for V in Plist for P in Plist if (V!=P)]

# we remove any variants for which the various answers aren't distinct:
L = [Q for Q in L if Q.has_distinct_answers()]

for Q in L:
    display(Q)
    
SaveToBBfile(L,'DirectionalDerivative2D.txt')

Here is a variant selection wizard, based on Jupyter Widgets. 

L must be a list of questions (produced above). The Wizard then displays the list and lets you select the variants 
you want to keep. This is useful if you want to throw out the easiest and hardest variants.

You can then write the Blackboard files and marking scheme to disk with the click of a button.

This is still work in progress, as evidenced by the To Do list, but it works for now.

In [ ]:
# Variant Selection Wizard
#
#
# To do:
#   - Improve layout of items
#   - Turn this into a function with input L and output selection
#   - Fix layout of descriptions
#   - Question classes need names + add "update filenames" button
#   - Show question type: written/MCQ
#   - Show Table headers
#   - Make this compatible with MCQ

b = 0

selection = []
items = [widgets.ToggleButton(description="variant "+str(i),button_style='') for i in range(len(L))]
master = widgets.VBox([widgets.HBox(
    [items[i], widgets.HTMLMath(value="   :   ".join([a for a in L[i].table_row]))]) for i in range(len(L))])
display(master)
# widgets.Label(value=str([c.value for c in items]))
out = widgets.Output(layout={'border': '1px solid black'},description='Status:')

Button_Count = widgets.Button(description="Count: ")
def Count(b):
    count=0
    for c in items:
        if c.value:
            count+=1
    Button_Count.description="Count: "+str(count)
Button_Count.on_click(Count)

def TB_on_value_change(change):
    Count(b)
    tb = change['owner']
    if tb.value:
        tb.icon='check'
    else:
        tb.icon=''

# The following might be reducing performance:
for tb in items:
    tb.observe(TB_on_value_change, names='value')

Button_SelectAll = widgets.Button(description="Select All")
def SelectAll(b):
    for c in items:
        c.value=true
    Button_Count.description="Count: "+str(len(L))
Button_SelectAll.on_click(SelectAll)

Button_SelectNone = widgets.Button(description="Select None")
def SelectNone(b):
    for c in items:
        c.value=false
    Button_Count.description="Count: 0"
Button_SelectNone.on_click(SelectNone)


Button_MakeSelection = widgets.Button(description="Make Selection")
def MakeSelection(b):
    selection = []
    count=0
    for i in range(len(L)):
        if items[i].value:
            selection.append(L[i])
            count+=1
    Button_Count.description="Count: "+str(count)
    with out:
        print("Selected %d Questions:"%count,[i for i in range(len(L)) if items[i].value])
    return(selection)
Button_MakeSelection.on_click(MakeSelection)

Button_ClearOutput = widgets.Button(description="Clear Output")
def ClearOutput(b):
    out.clear_output()
Button_ClearOutput.on_click(ClearOutput)

MarkingSchemeFilename = widgets.Text(value="Invert3x3-MarkingScheme.tex",description='MS Filename:')

MarkingSchemeArrayStretch = widgets.FloatText(value=1.5,step=0.1,description='Baseline_stretch:')

BBFilename = widgets.Text(value="Invert3x3.txt",description='BB filename:')

Path = widgets.Text(value="",description="File path:")

CourseName = widgets.Text(value="MATH1120 S2 2020", description="Course name:")

QuizTitle = widgets.Text(value="Workshop Quiz Week 11", description="Quiz name:")

MSPrintTable = widgets.Checkbox(value=true, description="Print table:")

Button_SaveMarkingScheme = widgets.Button(description="Save Marking Scheme")
def SaveMarkingSchemeB(b):
    selection=MakeSelection(b)
    if len(selection) == 0:
        with out:
            print('No variants selected!')
        return
    if Path.value!="":
        fn=Path.value+"\\"+MarkingSchemeFilename.value
    else:
        fn=MarkingSchemeFilename.value
    SaveMarkingScheme(selection,filename=fn,course=CourseName.value,title=QuizTitle.value,
                      print_table=MSPrintTable.value,array_stretch=MarkingSchemeArrayStretch.value)
    with out:
        print('Marking scheme for variants',[i for i in range(len(L)) if items[i].value],' saved to ',fn)
Button_SaveMarkingScheme.on_click(SaveMarkingSchemeB)

Button_PrintMarkingScheme = widgets.Button(description="Print Marking Scheme")
def PrintMarkingSchemeB(b):
    selection=MakeSelection(b)
    if len(selection) == 0:
        with out:
            print('No variants selected!')
        return    
    with out:
        print('Marking Scheme:')
        print()
        print(TypesetMarkingScheme(selection,course=CourseName.value,title=QuizTitle.value,
                                   array_stretch=MarkingSchemeArrayStretch.value))
Button_PrintMarkingScheme.on_click(PrintMarkingSchemeB)

Button_SaveBB = widgets.Button(description="Save Blackboard file")
def SaveBBFile(b):
    selection=MakeSelection(b)
    if len(selection) == 0:
        with out:
            print('No variants selected!')
        return    
    if Path!="":
        fn=BBFilename.value
    else:
        fn=BBFilename.value    
    SaveToBBfile(selection,fn)
    with out:
        print('BB entries for variants',[i for i in range(len(L)) if items[i].value],' saved to ',fn)
Button_SaveBB.on_click(SaveBBFile)

Button_PreviewQuestions = widgets.Button(description="Preview Selected Questions")
def PreviewQuestions(b):
    selection=MakeSelection(b)
    if len(selection) == 0:
        with out:
            print('No variants selected!')
        return
    with out:
        for Q in selection:
            display(Q)  
Button_PreviewQuestions.on_click(PreviewQuestions)

Button_PrintBBRows = widgets.Button(description="Print BB Rows")
def PrintBBRows(b):
    selection=MakeSelection(b)
    if len(selection) == 0:
        with out:
            print('No variants selected!')
        return
    with out:
        for Q in selection:
            print(Q.make_BB_row())
Button_PrintBBRows.on_click(PrintBBRows)

SelectRange = widgets.IntRangeSlider(
    value=[0, len(L)-1],
    min=0,
    max=len(L)-1,
    step=1,
    description='Select range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

Button_SelectAdd = widgets.Button(description="Add")
def SelectAdd(b):
    for i in range(SelectRange.value[0],SelectRange.value[1]+1):
        items[i].value=true
    Count(b)
Button_SelectAdd.on_click(SelectAdd)

Button_SelectRemove = widgets.Button(description="Remove")
def SelectRemove(b):
    for i in range(SelectRange.value[0],SelectRange.value[1]+1):
        items[i].value=false
    Count(b)
Button_SelectRemove.on_click(SelectRemove)

Button_SelectInvert = widgets.Button(description="Invert")
def SelectInvert(b):
    for i in range(SelectRange.value[0],SelectRange.value[1]+1):
        items[i].value=not items[i].value
    Count(b)
Button_SelectInvert.on_click(SelectInvert)

Selector_MasterMode = widgets.Dropdown(options=['Table Row','Question Preview'],value='Table Row',
                                     description='Master Mode')
def on_change_master_mode(change):
    if change['new']=='Table Row':
        for i in range(len(L)):
            master.children[i].children[1].value="   :   ".join([a for a in L[i].table_row])
    else:
        for i in range(len(L)):
            master.children[i].children[1].value=L[i].question_text
#         master = widgets.VBox([widgets.HBox(
#         [items[i], widgets.HTMLMath(value="   :   ".join([a for a in L[i].table_row]))]) for i in range(len(L))])
#     else:
#         master = widgets.VBox([widgets.HBox(
#         [items[i], widgets.HTMLMath(value=L[i].question_text)]) for i in range(len(L))])
Selector_MasterMode.observe(on_change_master_mode,names="value")

        
tab=widgets.Tab()
tab.children=[widgets.HBox([SelectRange,
                            Button_SelectAdd,
                            Button_SelectRemove,
                            Button_SelectInvert
                           ]),
              widgets.VBox([MarkingSchemeFilename, 
                            QuizTitle,
                            MSPrintTable,
                            MarkingSchemeArrayStretch,
                            widgets.HBox([Button_PrintMarkingScheme,Button_SaveMarkingScheme])
                           ]),
              widgets.VBox([BBFilename,
                            widgets.HBox([Button_PrintBBRows,Button_SaveBB])
                           ]),
              widgets.VBox([CourseName,
                            QuizTitle,
                            Path,
                            Selector_MasterMode,
                           ])
             ]
tab.set_title(index=0,title='Selection')
tab.set_title(index=1,title='Marking scheme')
tab.set_title(index=2,title='Blackboard')
tab.set_title(index=3,title='Options')



display(widgets.HBox([Button_Count,Button_SelectAll,Button_SelectNone,
                      Button_MakeSelection,Button_PreviewQuestions,Button_ClearOutput]))
display(tab)

display(out)
display(Button_ClearOutput)


Coming soon (hopefully): a widget-based question editor, which makes it easier to craft the question class itself. 

Everything below this line isn't quite ready of public consumption yet, and parts of it are broken. If you want to help develop this, please drop me a line.

In [ ]:
# replace bits of code with custom values
#

# First, we define a whole bunch of regular expressions:

# This contains too much boilerplate! Somebody should rewrite this in a small handful of methods.

pqname = re.compile(r'''
^
class[ ]
(?P<name>.*)
\(
''', re.VERBOSE|re.MULTILINE)

pqparameters = re.compile(r'''
\b
def[ ]__init__\(self,
(?P<params>.*)
,[ ]*variant_number=
''', re.VERBOSE)

pqquestion_text = re.compile(r'''
[ \t]*
self.question_text      #
[ ]*=[ ]*
(?P<question_text>.*?)   # the question text itself
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.MULTILINE|re.DOTALL)

pqsolution_text = re.compile(r'''
[ \t]*
self.solution_text      #
[ ]*=[ ]*
(?P<solution_text>.*?)   # the question text itself
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.MULTILINE|re.DOTALL)

re_finalslashplus = re.compile(r'''
\+\\
\s*
\Z
''', re.VERBOSE)

re_leading_whitespace = re.compile(r'''
^
[ \t\r\f\v]*
''', re.VERBOSE|re.MULTILINE)

re_code_block_1 = re.compile(r'''
variant_number[ ]*=[ ]*variant_number\n
(?:[ \t\r\f\v]*\n)    # one more newline 
(?P<code_block>.*?)
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.DOTALL)

re_table_header = re.compile(r'''
table_header[ ]*=[ ]*
(?P<header>.*?)
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.DOTALL)

re_table_row = re.compile(r'''
table_row[ ]*=[ ]*
(?P<row>.*?)
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.DOTALL)

re_answer_text = re.compile(r'''
self[.]answer[ ]*=[ ]*
(?P<answer>.*?)
(?:[ \t\r\f\v]*\n){2} # ends when we find two newlines
''', re.VERBOSE|re.DOTALL)


# Next, a bunch of methods that use these re's to replace parts of the CodeText

def read_answer_text(s):
    '''Returns the answer_text in s (MCQ)'''
    return(re_answer_text.search(s).group('answer'))

def write_answer_text(s, new_answer):
    '''Writes new_answer to answer_text in s, returns the result.'''
    t = re_answer_text.sub('self.answer = PLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',new_answer+'\n\n')
    return(t)

def read_table_header(s):
    '''Returns the table_header in s'''
    return(re_table_header.search(s).group('header'))

def write_table_header(s, new_header):
    '''Writes new_header to table_header in s, returns the result. '''
    # here we first use pqquestion_text.sub() to find the right place
    # but then insert the actual new_text using string.replace(), which doesn't trigger escapes
    t = re_table_header.sub('table_header = PLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',new_header+'\n\n')
    return(t)

def read_table_row(s):
    '''Returns the table_row in s'''
    return(re_table_row.search(s).group('row'))

def write_table_row(s, new_row):
    '''Writes new_row to table_row in s, returns the result. '''
    # here we first use pqquestion_text.sub() to find the right place
    # but then insert the actual new_text using string.replace(), which doesn't trigger escapes
    t = re_table_row.sub('table_row = PLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',new_row+'\n\n')
    return(t)

def read_code_block_1(s):
    '''Returns code block 1 in s, between variant_number and question_text'''
    return(re_code_block_1.search(s).group('code_block'))

def write_code_block_1(s, new_code):
    '''Writes code block from new_code to s, between variant_number and question_text, returns the result.'''
    # here we first use pqquestion_text.sub() to find the right place
    # but then insert the actual new_text using string.replace(), which doesn't trigger escapes
    t = re_code_block_1.sub('variant_number = variant_number\n\nPLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',new_code+'\n\n')
    return(t)


def add_indent(s, n=1, not_first_line=false):
    '''add n tabs indent to s'''
    tabs = ' '*(4*n)
    if not_first_line:
        return(s.replace('\n','\n'+tabs))
    else:
        return(tabs+s.replace('\n','\n'+tabs))

def set_indent(s, n=1, not_first_line=false):
    '''Sets the indent of to n'''
    tabs = ' '*(4*n)
    s = re_leading_whitespace.sub('',s)
    if not_first_line:
        return(s.replace('\n','\n'+tabs))
    else:
        return(tabs+s.replace('\n','\n'+tabs))

def WriteName(s, new_name):
    '''replaces the name of the class in s with newname'''
    return(pqname.sub(r'class '+new_name+r'(',s,count=1))

def ReadName(s):
    '''returns the name of the class'''
    return(pqname.search(s).group('name'))

def WriteParameters(s, new_params):
    '''replaces the parameters of __init__ in s with newparams'''
    return(pqparameters.sub(r'def __init__(self,'+new_params+',variant_number=',s,count=1))

def ReadParameters(s):
    '''returns the parameter list'''
    return(pqparameters.search(s).group('params'))

# def WriteQuestionText(s, new_text, tabs=2):
#     ''' replaces question_text with newtext'''
#     return(pqquestion_text.sub(set_indent('self.question_text = \\\n'
#                                + new_text, n=tabs) + '\n\n', s, count=1))

def WriteQuestionText(s, new_text, tabs=2):
    ''' replaces question_text with newtext'''
    # here we first use pqquestion_text.sub() to find the right place
    # but then insert the actual new_text using string.replace(), which doesn't trigger escapes
    t = pqquestion_text.sub(' '*(4*tabs)+'self.question_text = \\\nPLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',set_indent(new_text, n=tabs)+'\n')
    return(t)


def ReadQuestionText(s):
    '''returns the question_text'''
    return(pqquestion_text.search(s).group('question_text'))

def WriteSolutionText(s, new_text, tabs=2):
    ''' replaces question_text with newtext'''
    # here we first use pqquestion_text.sub() to find the right place
    # but then insert the actual new_text using string.replace(), which doesn't trigger escapes
    t = pqsolution_text.sub(' '*(4*tabs)+'self.solution_text = \\\nPLACEHOLDER', s, count=1).replace(
        'PLACEHOLDER',set_indent(new_text, n=tabs)+'\n')
    return(t)

# def WriteSolutionText(s, new_text, tabs=2):
#     ''' replaces question_text with newtext'''
#     return(pqsolution_text.sub(set_indent('self.solution_text = \\\n'
#                                + new_text, n=tabs) + '\n\n', s, count=1))

def ReadSolutionText(s):
    '''returns the question_text'''
    return(pqsolution_text.search(s).group('solution_text'))


In [ ]:
# Translate text composed in LaTeX with \sage{}, \begin{sage}\end{sage} etc delimiters
# into string constructor for sage

pqsage = re.compile(r'''
\\begin{sage}    # enter sage mode 
\s*              # eat whitespaces
(.*?)            # stuff in sage mode
\s*              # eat whitespaces
\\end{sage}      # exit sage mode
|                # or
\\sage{          # enter sage mode
\s*              # eat whitespaces
(.*?)            # stuff in sage mode
\s*              # eat whitespaces
}                # exit sage mode
''',re.VERBOSE|re.DOTALL)

pqsagel = re.compile(r'''
\\begin{sagel}    # enter sage mode
\s*              # eat whitespaces
(.*?)             # stuff in sage mode
\s*              # eat whitespaces
\\end{sagel}      # exit sage mode
|                 # or 
\\sagel{          # enter sage mode
\s*              # eat whitespaces
(.*?)             # stuff in sage mode
\s*              # eat whitespaces
}                 # exit sage mode
''',re.VERBOSE|re.DOTALL)

def pqtranslate(s):
    '''Translates string s from latex/sage code to a sage string constuctor'''
    # first kill any leading whitespace
    s = re.sub(r'^\s*','',s)
    if s=='':
        return('""')
    L = pqsagel.split(s)  # split according to \sagel{} and \begin{sagel} \end{sagel}
    L = [l for l in L if l is not None]  # Remove Nones
    # Odd entries of L are sage code, to be encapsulated in +latex()+\   :
    L[1::2]=list(map(lambda s: "+latex("+s+")+\\", L[1::2]))
    # even entries must be split further:
    Lnew = []
    for i in range(len(L)):
        if i%2==0:               # even entry; should be processed further
            L2 = pqsage.split(L[i])   # split according to \sage{} and \begin{sage} \end{sage}
            L2 = [l for l in L2 if l is not None]  # Remove Nones
            # L2's odd entries are sage code, to be encapsulated in + +\:
            L2[1::2]=list(map(lambda s: "+"+s+"+\\", L2[1::2]))  
            Lnew+=L2
        else:
            Lnew.append(L[i])
    L=Lnew
    # print("*** L = ",L)  # for debugging
    # now L's even entries should be strings, and odd entries processed sage code
    for i in range(0,len(L),2):
        LL = L[i].split('\n')
        if LL[-1] == "":
            LL.pop()   # remove the last empty string from LL, if there is one
        L[i] = ''.join(['r"'+ss+r'"+"\n"'+'\\\n' for ss in LL]).replace('r""','')
    s = ''.join(L)
    # print(L)   # for debugging
    # Need to clean up some more, e.g. strip any trailing + and/or \
    s = re_finalslashplus.sub('',s)
    s = s+'\n\n'
    return(s)

moretest = \
r'''Easy: just do this:
\begin{sagel}
f'(x)=3x^2
\end{sagel}'''

#print(set_indent(pqtranslate(moretest),4))
print(pqtranslate(moretest))
pqtranslate(moretest)

In [ ]:
# Let's start a new question editor!
#
# To do:
# - Read from file
# - checkbox to unlink filename from question name
# - file overwrite warning first time we write to file
# - "Run code" should save to file first
# - implement MCQ: wrong answers
# - Create variation, link to variation selector, put in accordion?
# - Comply with PEP8
# - Undo button
# - make edit box 3 buttons wide
# - update question name in preview code
# - make preview code editor with some templates
# - toggle edit / syntax highlighting in code boxes, use title Label as button
# - latex syntax highlighting in out?
# - preview solution_page by calling SumatraPDF?



start_with_MCQ = true

# Templates:
template_written = \
'class Question_Name(Question_Written):'\
'\n'\
'    def __init__(self, a=0, b=0, variant_number=0):\n'\
'        Question_Written.__init__(self)\n'\
'        self.variant_number = variant_number\n'\
'\n'\
'        # Code Block 1:\n'\
'\n'\
'        self.question_text=""\n'\
'\n'\
'        self.solution_text=""\n'\
'\n'\
'        self.table_header = []\n'\
'\n'\
'        self.table_row = []\n'\
'\n'\
'Q = Question_Name()\n'\
'display(Q)'

template_MCQ = \
'class Question_Name(Question_MCQ):'\
'\n'\
'    def __init__(self, a=0, b=0, variant_number=0):\n'\
'        Question_Written.__init__(self)\n'\
'        self.variant_number = variant_number\n'\
'\n'\
'        # Code Block 1:\n'\
'\n'\
'        self.question_text=""\n'\
'\n'\
'        self.answer=""\n'\
'\n'\
'        self.wrong_answers = []\n'\
'\n'\
'\n'\
'Q = Question_Name()\n'\
'display(Q)'

if start_with_MCQ:
    CodeText = template_MCQ
else:
    CodeText = template_written


# information-containig widgets:
MainCodeHTML = widgets.HTML(value=highlight(CodeText, PythonLexer(), HtmlFormatter(full=true)), 
                        layout={'border': '1px solid black', 'width': '80ex', 'height': '150ex'})
MainCodeEditor = widgets.Textarea(value=CodeText, 
                                  layout={'border': '1px solid black', 
                                          'width': '80ex', 
                                          'height': '150ex', 
                                          'font_family' : 'Monospace'})   # unfortunately, Jupyter ignores the font
MainCodeBox = widgets.HBox([MainCodeHTML])   # its children can be edited dynamically

QuestionType = widgets.Dropdown(options=['MCQ','Written'], value='Written', description='Question Type')
if start_with_MCQ:
    QuestionType.value='MCQ'
QuestionName = widgets.Text(value='Question_Name',description='Q Name')
QuestionParams = widgets.Text(value='a=0, b=0',description='Parameters')
QuestionText = widgets.Textarea(value='',
                                layout=Layout( width='auto', height='20ex'), 
                                style={'font_family':'Courier New'})
SolutionText = widgets.Textarea(value='',layout=Layout( width='auto', height='20ex'))
FileName = widgets.Text(value='Question_name.sage', description='Filename')
code_block_editor = widgets.Textarea(placeholder='Type some code here',  
                                    layout=Layout( width='auto', height='20ex'))
table_header_editor = widgets.Textarea(placeholder='[ ]', layout=Layout( width='auto', height='20ex'))
table_row_editor = widgets.Textarea(placeholder='[ ]', layout=Layout( width='auto', height='20ex'))
answer_text_editor = widgets.Textarea(value='',layout=Layout( width='auto', height='20ex'))

# Output:
out = widgets.Output(layout={'border': '1px solid black'},description='Status:')
# out_preview = widgets.Output(layout={'border': '1px solid black'},description='Status:')

# Buttons:
# UpdateCode = widgets.Button(description='Update code', button_style='success')
# ReadCode = widgets.Button(description='Read code', button_style='info')
ClearOutput = widgets.Button(description='Clear output')
PreviewQuestion = widgets.Button(description='Preview Question', button_style='success')
SaveCode = widgets.Button(description='Save to File', button_style='danger')
RunCode = widgets.Button(description='Run Code', button_style='success')
insert_question_text_button = widgets.Button(description='Insert', button_style='success')
translate_question_text_button = widgets.Button(description='Translate',button_style='info')
insert_solution_text_button = widgets.Button(description='Insert', button_style='success')
translate_solution_text_button = widgets.Button(description='Translate',button_style='info')
toggle_main_code_button = widgets.ToggleButton(description='Edit Code',value=false)
read_code_block_1_button = widgets.Button(description='Read code block 1', button_style='info')
insert_code_block_1_button = widgets.Button(description='insert code block 1', button_style='info')
read_table_header_button = widgets.Button(description='Read header row')
read_table_row_button = widgets.Button(description='Read table row')
insert_table_header_button = widgets.Button(description='Insert header row')
insert_table_row_button = widgets.Button(description='Insert table row')
paste_template_button = widgets.Button(description='Paste Template', button_style='danger')
insert_header_button = widgets.Button(description='Paste Header')
read_header_button = widgets.Button(description='Read Header')
read_file_button = widgets.Button(description='Load File')
insert_answer_text_button = widgets.Button(description='Insert', button_style='success')
translate_answer_text_button = widgets.Button(description='Translate',button_style='info')

# Button code:
def click_insert_answer_text_button(b):
    MainCodeEditor.value = write_answer_text(CodeText, answer_text_editor.value)
insert_answer_text_button.on_click(click_insert_answer_text_button)
    
def click_translate_answer_text_button(b):
    MainCodeEditor.value = write_answer_text(CodeText, pqtranslate(answer_text_editor.value))
translate_answer_text_button.on_click(click_translate_answer_text_button)

def click_insert_header_button(b):
    global CodeText
    CodeText = WriteName(CodeText, QuestionName.value)
    CodeText = WriteParameters(CodeText, QuestionParams.value)
    MainCodeEditor.value = CodeText
insert_header_button.on_click(click_insert_header_button)

def click_read_header_button(b):
    global CodeText
    QuestionName.value = ReadName(CodeText)
    QuestionParams.value = ReadParameters(CodeText) 
read_header_button.on_click(click_read_header_button)

def click_paste_template_button(b):
    if QuestionType.value == 'MCQ':
        MainCodeEditor.value = template_MCQ
    elif QuestionType.value == 'Written':
        MainCodeEditor.value = template_written
paste_template_button.on_click(click_paste_template_button)

def click_read_table_header_button(b):
    table_header_editor.value = read_table_header(CodeText)
read_table_header_button.on_click(click_read_table_header_button)
    
def click_insert_table_header_button(b):
    MainCodeEditor.value = write_table_header(CodeText, set_indent(table_header_editor.value, 2, not_first_line=true))
insert_table_header_button.on_click(click_insert_table_header_button)

def click_read_table_row_button(b):
    table_row_editor.value = read_table_row(CodeText)
read_table_row_button.on_click(click_read_table_row_button)
    
def click_insert_table_row_button(b):
    MainCodeEditor.value = write_table_row(CodeText, set_indent(table_row_editor.value, 2, not_first_line=true))
insert_table_row_button.on_click(click_insert_table_row_button)

def click_read_code_block_1_button(b):
    code_block_editor.value = set_indent(read_code_block_1(CodeText),0)
read_code_block_1_button.on_click(click_read_code_block_1_button)

def click_insert_code_block_1_button(b):
    MainCodeEditor.value = write_code_block_1(CodeText, set_indent(code_block_editor.value, 2))
insert_code_block_1_button.on_click(click_insert_code_block_1_button)

def click_toggle_main_code_button(b):
    if toggle_main_code_button.value:
        toggle_main_code_button.icon = 'check'
        MainCodeBox.children = (MainCodeEditor,)
    else:
        toggle_main_code_button.icon = ''
        MainCodeHTML.value=highlight(CodeText, PythonLexer(), HtmlFormatter(full=true))
        MainCodeBox.children = (MainCodeHTML,)
toggle_main_code_button.observe(click_toggle_main_code_button, names='value')

def click_insert_question_text_button(b):
    MainCodeEditor.value = WriteQuestionText(CodeText, QuestionText.value)
insert_question_text_button.on_click(click_insert_question_text_button)

def click_translate_question_text_button(b):
    MainCodeEditor.value = WriteQuestionText(CodeText, pqtranslate(QuestionText.value))
translate_question_text_button.on_click(click_translate_question_text_button)

def click_insert_solution_text_button(b):
    MainCodeEditor.valuet = WriteSolutionText(CodeText, SolutionText.value)
insert_solution_text_button.on_click(click_insert_solution_text_button)

def click_translate_solution_text_button(b):
    MainCodeEditor.value = WriteSolutionText(CodeText, pqtranslate(SolutionText.value))
translate_solution_text_button.on_click(click_translate_solution_text_button)

# def Click_UpdateCode(b):
#     global CodeText
#     CodeText = WriteName(CodeText,QuestionName.value)
#     CodeText = WriteParameters(CodeText,QuestionParams.value)
#     CodeText = WriteQuestionText(CodeText,pqtranslate(QuestionText.value))
#     CodeText = WriteSolutionText(CodeText,pqtranslate(SolutionText.value))
#     MainCodeHTML.value = highlight(CodeText, PythonLexer(), HtmlFormatter(full=true))
#     MainCodeEditor.value = CodeText
#     with out:
#         print('question_text = ',pqtranslate(QuestionText.value),'\n')
#         print('solution_text = ',pqtranslate(SolutionText.value),'\n')
# UpdateCode.on_click(Click_UpdateCode)

# def Click_ReadCode(b):
#     with out:
#         print('question_text = ',ReadQuestionText(CodeText),'\n')
#         print('solution_text = ',ReadSolutionText(CodeText))
#     QuestionName.value = ReadName(CodeText)
#     QuestionParams.value = ReadParameters(CodeText) 
# ReadCode.on_click(Click_ReadCode)

def Click_ClearOutput(b):
    out.clear_output()
ClearOutput.on_click(Click_ClearOutput)

# def Click_PreviewQuestion(b):
#     with out:
#         print("I'm doing something!")
# #     global get_ipython
# #     get_ipython().set_next_input(CodeText)
# PreviewQuestion.on_click(Click_PreviewQuestion)

def Click_SaveCode(b):
    with open(FileName.value,'w') as f:
        f.write(CodeText)
SaveCode.on_click(Click_SaveCode)

def Click_RunCode(b):
    Click_SaveCode(b)  # first write code to file
    with out:
        load(FileName.value)  # then load it again
RunCode.on_click(Click_RunCode)

def click_read_file_button(b):
    global CodeText
    with open(FileName.value,'r') as f:
        CodeText = f.read()
    MainCodeEditor.value = CodeText
    QuestionName.value = ReadName(CodeText)
    QuestionParams.value = ReadParameters(CodeText)
    # read question type - to be implemented
read_file_button.on_click(click_read_file_button)


# Observe code:

# Whenever MainCodeEditor changes, update CodeText and MainCodeHTML
# This means that all we ever need to change is MainCodeEditor.value
def on_change_MainCodeEditor(change):
    global CodeText
    CodeText = MainCodeEditor.value
    MainCodeHTML.value = highlight(CodeText, PythonLexer(), HtmlFormatter(full=true))
MainCodeEditor.observe(on_change_MainCodeEditor, names='value')    
   
def on_change_question_type(change):
    if QuestionType.value == 'MCQ':
        edit_box.children = (edit_MCQ,)
    elif QuestionType.value =='Written':
        edit_box.children = (edit_written_question, )
QuestionType.observe(on_change_question_type, names='value')
        
    
# Main layout:
# Main_Tab = widgets.Tab()
# Main_Tab.children = [

edit_MCQ = widgets.VBox([
    widgets.Label(value='MCQ coming soon!')
])
edit_written_question = widgets.VBox([
        widgets.Label(value='Edit code block 1:'), 
        code_block_editor,
        widgets.HBox([read_code_block_1_button, insert_code_block_1_button]),
        widgets.Label(value='Edit the question text, or paste from LaTeX file:'),
        QuestionText,
        widgets.HBox([insert_question_text_button, translate_question_text_button]),
        widgets.Label(value='Edit the solution text, or paste from LaTeX file:'),
        SolutionText,
        widgets.HBox([insert_solution_text_button, translate_solution_text_button]),
        widgets.Label(value='Edit table header:'), 
        table_header_editor,
        widgets.HBox([read_table_header_button, insert_table_header_button]),
        widgets.Label(value='Edit table row:'), 
        table_row_editor,
        widgets.HBox([read_table_row_button, insert_table_row_button]),  
])
edit_MCQ = widgets.VBox([
        widgets.Label(value='Edit code block 1:'), 
        code_block_editor,
        widgets.HBox([read_code_block_1_button, insert_code_block_1_button]),
        widgets.Label(value='Edit the question text, or paste from LaTeX file:'),
        QuestionText,
        widgets.HBox([insert_question_text_button, translate_question_text_button]),
        widgets.Label(value='Edit the answer text, or paste from LaTeX file:'),
        answer_text_editor,
        widgets.HBox([insert_answer_text_button, translate_answer_text_button]),
])

if start_with_MCQ:
    edit_box = widgets.Box((edit_MCQ,))
else:
    edit_box = widgets.Box((edit_written_question,)) # this can be changed by selecting MCQ...
    
main_box = widgets.HBox([  # Edit Code Tab
    widgets.VBox([
        QuestionType,
        QuestionName,
        QuestionParams,
        widgets.HBox([read_header_button, insert_header_button]),
        edit_box,
#                             widgets.HBox([UpdateCode, ReadCode]),
        FileName,
        widgets.HBox([SaveCode, read_file_button]),
        widgets.HBox([RunCode, ClearOutput]),
    ]),
    widgets.VBox([
        widgets.HBox([toggle_main_code_button, paste_template_button]),
        MainCodeBox,
    ]),
])

# Main display:
display(main_box)
display(out)
display(ClearOutput)